In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.metrics import classification_report,confusion_matrix
import os
import random
import shutil
from shutil import copyfile
from keras.preprocessing.image import ImageDataGenerator
import cv2

In [2]:
values = ['crop','weed']

In [3]:
dest_dir='./dataset/'
train_dir=dest_dir+'training/'
val_dir=dest_dir+'validation/'

In [4]:

# Specify the GPU device to use
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Restrict TensorFlow to use only the first GPU
    tf.config.set_visible_devices(gpus[0], 'GPU')
    # Allow memory growth to prevent out-of-memory errors
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print('Configured TensorFlow to use GPU:', gpus[0])
  except RuntimeError as e:
    print(e)
else:
  print('No GPU detected, using CPU instead.')

# Your TensorFlow code here...


Configured TensorFlow to use GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


Image Data Generator

In [5]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
  train_datagen = ImageDataGenerator(rescale=1/255)
  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=512,
                                                      class_mode='categorical',
                                                      target_size=(200,200))
 
  validation_datagen = ImageDataGenerator(rescale=1/255,
                                    #  rotation_range=40,
                                    #  width_shift_range=0.2,
                                    #  height_shift_range=0.2,
                                    #  shear_range=0.2,
                                    #  zoom_range=0.2,
                                    #  horizontal_flip=True,
                                    #  fill_mode='nearest'
                                     )
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=512,
                                                                class_mode='categorical',
                                                                target_size=(200,200))
  return train_generator, validation_generator

In [6]:
train_generator, validation_generator = train_val_generators(train_dir,val_dir)

Found 1040 images belonging to 2 classes.
Found 260 images belonging to 2 classes.


Model building

In [7]:
#parameters
img_height=512
img_width=512
num_classes=2
do=0.5
reg=tf.keras.regularizers.l2(l2=0.0015)
batch_size=128
epochs=30

In [18]:
def create_model():
  model = Sequential([
    Conv2D(32, 3 , activation='relu',input_shape=(img_height, img_width, 3)),
    Conv2D(32, 3 ,  activation='relu',padding='same'),
    MaxPooling2D(pool_size=4),
    #Dropout(do),

    Conv2D(64, 3 ,  activation='relu',padding='same'),
    Conv2D(64, 3 ,  activation='relu'),
    MaxPooling2D(pool_size=4),
    #Dropout(do),

    Conv2D(64, 3 ,  activation='relu',padding='same'),
    Conv2D(64, 3 ,  activation='relu'),
    MaxPooling2D(pool_size=4),
    #Dropout(do),

    Flatten(),
    Dense(512, activation='relu'),
    # Dropout(2*do),
    Dense(num_classes, activation='softmax')
    ])
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

  return model

In [19]:
model=create_model()

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 510, 510, 32)      896       
                                                                 
 conv2d_13 (Conv2D)          (None, 510, 510, 32)      9248      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 127, 127, 32)     0         
 2D)                                                             
                                                                 
 conv2d_14 (Conv2D)          (None, 127, 127, 64)      18496     
                                                                 
 conv2d_15 (Conv2D)          (None, 125, 125, 64)      36928     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 31, 31, 64)       0         
 2D)                                                  

In [21]:
callbacks=keras.callbacks.ModelCheckpoint('./../checkpoints/checkpoint-{epoch}.h5', save_best_only=True)

In [22]:
history = model.fit(
    train_generator,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=validation_generator,
    verbose=2, #if hvd.rank() == 0 else 0,
    callbacks=callbacks
)

Epoch 1/30


InvalidArgumentError: Graph execution error:

Detected at node 'sequential_2/flatten_2/Reshape' defined at (most recent call last):
    File "c:\Users\haris\miniconda3\envs\ml\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\haris\miniconda3\envs\ml\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\Users\haris\AppData\Roaming\Python\Python39\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\haris\AppData\Roaming\Python\Python39\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "C:\Users\haris\AppData\Roaming\Python\Python39\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "C:\Users\haris\AppData\Roaming\Python\Python39\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Us